# ipywidgets-jsonschema Quick Demo

In [ ]:
# This notebook shows:
# 1. Generating a form from a raw JSON Schema
# 2. Generating a form from a Pydantic model
# 3. Nested models, lists, enums
# 4. Customizing layout and options
# 5. Interactive editing via `PydanticEditorMixin`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import List, Dict, Union, Optional
from enum import Enum
from pydantic import BaseModel, Field
from ipywidgets_jsonschema import Form, PydanticEditorMixin
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# 1. Raw JSON Schema

# Define a plain JSON Schema and render a form:

In [ ]:
person_schema = {
    "title": "Person",
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Your full name"},
        "age": {"type": "integer", "minimum": 0, "description": "Your age in years"},
        "email": {
            "type": "string",
            "format": "email",
            "description": "Optional email address",
        },
    },
    "required": ["name", "age"],
}

In [ ]:
form1 = Form(person_schema, show_descriptions=True)
form1.show()

In [ ]:
# 2. Pydantic Model

# You can pass a Pydantic `BaseModel` class directly to `Form`, which will internally convert it to JSON Schema.

In [ ]:
class Person(BaseModel):
    name: str = Field(..., description="Your full name")
    age: int = Field(..., ge=0, description="Your age in years")
    email: Optional[str] = Field(None, description="Email address")


form2 = Form(
    Person, vertically_place_labels=True, use_sliders=True, show_descriptions=True
)
form2.show()

In [ ]:
# 3. Nested Models, Lists & Dicts

In [ ]:
class Address(BaseModel):
    street: str
    city: str
    zipcode: str = Field(..., description="5-digit ZIP code")


class Employee(BaseModel):
    name: str
    address: Address
    tags: List[str] = Field(default_factory=list, description="List of tags")
    metadata: Dict[str, Union[str, int]] = Field(default_factory=dict)


# Prebuild two list entries to show
form3 = Form(Employee, preconstruct_array_items=2, show_descriptions=True)
form3.show()

In [ ]:
# 4. Enums & Union Types

In [ ]:
class Role(Enum):
    ADMIN = "Admin"
    USER = "User"
    GUEST = "Guest"


class Member(BaseModel):
    username: str
    role: Role = Field(..., description="Select your role")
    extra: Union[str, int] = Field(..., description="Accepts string or integer")


form4 = Form(Member, show_descriptions=True)
form4.show()

In [ ]:
# 5. Interactive Editing with `PydanticEditorMixin`

# Mix your Pydantic model with `PydanticEditorMixin` to get an “Edit → Save/Cancel” UI.

In [ ]:
class User(PydanticEditorMixin, BaseModel):
    name: str
    age: int
    email: Optional[str]


# instantiate and launch editor
user = User(name="Alice", age=28, email="alice@example.com")
user.edit()